# Chicago Yelp API

### imports


In [59]:
import requests
import pandas as pd
import time

### load api key

In [62]:
with open("api_key_yelp.txt", "r") as file:
    api_key = file.read().strip()

### header and params

In [65]:
headers = {"Authorization": f"Bearer {api_key}"}
url = "https://api.yelp.com/v3/businesses/search"

In [67]:
params = {"location": "Chicago", "term": "food", "limit": 50}

### send req

In [81]:
businesses = []
#for error "limit+offset must be <= 240."
total = 240
offset = 0
while offset < total:
    params["offset"] = offset
    response = requests.get(url, headers=headers, params=params)
    # status ok
    if response.status_code == 200:
        data = response.json()
        businesses.extend(data.get("businesses",[]))
    else:
        print({response.status_code})
        print(response.text)
        break
    offset += params["limit"]
    time.sleep(1)

{400}
{"error": {"code": "VALIDATION_ERROR", "description": "Too many results requested, limit+offset must be <= 240."}}


### get df

In [91]:
df = pd.DataFrame(all_businesses)

In [78]:
print(response.status_code)
print(df.shape[0])

400
200


## I get 200 rows max and 400 status error

In [89]:
df.head(200)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes
0,qjnpkS8yZO8xcyEIy5OU9A,girl-and-the-goat-chicago,Girl & The Goat,https://s3-media1.fl.yelpcdn.com/bphoto/ya6gjD...,False,https://www.yelp.com/biz/girl-and-the-goat-chi...,10417,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,"{'latitude': 41.88419277030351, 'longitude': -...",[delivery],$$$,"{'address1': '809 W Randolph', 'address2': '',...",+13124926262,(312) 492-6262,3395.120160,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
1,boE4Ahsssqic7o5wQLI04w,the-purple-pig-chicago,The Purple Pig,https://s3-media3.fl.yelpcdn.com/bphoto/rHHvhR...,False,https://www.yelp.com/biz/the-purple-pig-chicag...,8774,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.3,"{'latitude': 41.890694, 'longitude': -87.624782}","[delivery, pickup]",$$$,"{'address1': '444 N Michigan Ave', 'address2':...",+13124641744,(312) 464-1744,4670.876907,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
2,LPwAwxEjetjdNh7Uadro3g,smoque-bbq-chicago,Smoque BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/9n0qsF...,False,https://www.yelp.com/biz/smoque-bbq-chicago?ad...,4863,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.3,"{'latitude': 41.95021, 'longitude': -87.72795}",[],$$,"{'address1': '3800 N Pulaski Rd', 'address2': ...",+17735457427,(773) 545-7427,6500.229681,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
3,xJEA6P6VS-Ukl9dmjleDkw,silli-kori-chicago,Silli Kori,https://s3-media1.fl.yelpcdn.com/bphoto/CYGJvK...,False,https://www.yelp.com/biz/silli-kori-chicago?ad...,368,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.4,"{'latitude': 41.90286, 'longitude': -87.67935}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '2053 W Division St', 'address2':...",+17733845352,(773) 384-5352,281.552094,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
4,ZhHonGY7XknrPjwtx2jg2A,khmai-chicago,Khmai,https://s3-media2.fl.yelpcdn.com/bphoto/_0__SH...,False,https://www.yelp.com/biz/khmai-chicago?adjust_...,50,"[{'alias': 'cambodian', 'title': 'Cambodian'}]",4.6,"{'latitude': 42.00239984854286, 'longitude': -...",[],NaN,"{'address1': '6580 N Sheridan Rd', 'address2':...",+13126267710,(312) 626-7710,10901.348559,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,HiCXpGgxqjHPIlLyoZ06yw,carniceria-maribel-chicago,Carniceria Maribel,https://s3-media2.fl.yelpcdn.com/bphoto/qbrsZA...,False,https://www.yelp.com/biz/carniceria-maribel-ch...,160,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",4.8,"{'latitude': 41.85201, 'longitude': -87.67111}",[delivery],$$,"{'address1': '1801 W Cermak Rd', 'address2': '...",+17732546446,(773) 254-6446,5922.710922,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': Non..."
196,tZw353aqH3YYVkx_gKis4g,gold-ladle-chicago,Gold Ladle,https://s3-media1.fl.yelpcdn.com/bphoto/jpjAjp...,False,https://www.yelp.com/biz/gold-ladle-chicago?ad...,28,"[{'alias': 'thai', 'title': 'Thai'}]",4.7,"{'latitude': 41.9181596, 'longitude': -87.6774...","[pickup, delivery]",NaN,"{'address1': '2011 N Damen Ave', 'address2': N...",+17739691921,(773) 969-1921,1445.840889,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non..."
197,GIswPZXoc7dR8Z6XbIyOdw,chavas-tacos-el-original-chicago,Chavas Tacos El Original,https://s3-media3.fl.yelpcdn.com/bphoto/21fByU...,False,https://www.yelp.com/biz/chavas-tacos-el-origi...,281,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.6,"{'latitude': 41.890598, 'longitude': -87.685539}","[pickup, delivery]",$,"{'address1': '